In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma


/home/syedusmani/Documents/Projects/RAG_Assistant/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
vector_store = Chroma(embedding_function=embeddings, persist_directory="../chroma_db")

query_engine = vector_store.as_retriever(search_kwargs={"k": 4})
print("Retriever is ready. Connected to ChromaDB.")

Retriever is ready. Connected to ChromaDB.


In [3]:
test_docs = query_engine.invoke("How does auto-context work?")
print(f"Retrieved {len(test_docs)} documents.")
for doc in test_docs:
    print(f"From: {doc.metadata.get('source', 'N/A')}")
    print(doc.page_content[:200])  # First 200 chars
    print("---")

Retrieved 4 documents.
From: data/ADVANCED_ML_ALGORITHMS/Auto-context and Its Application to High-level Vision Tasks.pdf
# Auto-context and Its Application to High-level Vision Tasks and 3D Brain Image Segmentation  
Zhuowen Tu and Xiang Bai  
Lab of Neuro Imaging, University of California, Los Angeles  
_{_ ztu,xiang.b
---
From: data/ADVANCED_ML_ALGORITHMS/Efficient Scale Space Auto-Context for Image Segmentation and Labeling.pdf
## **Efficient Scale Space Auto-Context for Image Segmentation and Labeling**  
## Jiayan Jiang and Zhuowen Tu  
Lab of Neuro Imaging, Department of Neurology, Department of Computer Science, UCLA _{_
---
From: data/DEEP_LEARNING/Transformer_07-2.pdf
# Residual Connections  
OMITTED IMAGE  
• Each encoder has a residual connection, followed by  
layer normalization  
## Non-Autoregressive Encoding  
OMITTED IMAGE  
## Autoregressive Decoding  
OMI
---
From: data/DEEP_LEARNING/CNN_04-1.pdf
# _if we want to recognize even a few basic patterns at each location,_

In [14]:
from langchain.tools import tool
@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=5)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [15]:
from langchain.agents import create_agent
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.1:8b")
prompt = (
    "You have access to a tool that retrieves context from a school readings and lectures. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model=llm, tools=[retrieve_context], system_prompt=prompt)

In [16]:
query = (
    "what is Monte Carlo Dropout and why is it used?"
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

what is Monte Carlo Dropout and why is it used?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (d174e36a-36e9-4ec1-bbc0-b20980a8e93b)
 Call ID: d174e36a-36e9-4ec1-bbc0-b20980a8e93b
  Args:
    query: Monte Carlo Dropout and its purpose
================================= Tool Message =================================
Name: retrieve_context

Source: {'header': '', 'chunk_number': 0, 'source': 'data/AI_ALGORITHMS/06_MonteCarlo.pdf', 'course': 'AI_ALGORITHMS'}
Content: **Monte Carlo methods "I'd rather be vaguely right than precisely wrong." — John Maynard Keynes l**  
**“Like an octopus under a rock, Le Chiffre watched him from the other side of the table. Bond reached out a steady right hand and drew the cards towards him.” — Ian Fleming,** _**Casino Royale**_ **l**  
**Jason Fleischer, PhD Department of Cognitive Science, UC San Diego https://